<a href="https://colab.research.google.com/github/inuin09/Natural-language-Processing/blob/main/%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 맨 처음 부분 복습

In [ ]:
#기초설치
!apt-get update
!apt-get install g++ openjdk-8-jdk
!pip install JPype1
!pip install rhinoMorph

In [3]:
#RHINO 시작
import rhinoMorph
rn = rhinoMorph.startRhino()  #형태소분석기 객체를 rn이라는 이름으로 생성

filepath:  /usr/local/lib/python3.7/dist-packages
classpath:  /usr/local/lib/python3.7/dist-packages/rhinoMorph/lib/rhino.jar
RHINO started!


In [4]:
#예문 분석
text='한글 테스트 글을 남겨주세요'
sample_data = rhinoMorph.onlyMorph_list(rn, text)
print('sample data : ', sample_data)

sample data :  ['한글', '테스트', '글', '을', '남기', '어', '주', '시', '어요']


In [6]:
#RHINO 사용
import rhinoMorph
rn=rhinoMorph.startRhino()
text = '한글로 된 한글텍스트를 분석하는 것은 즐겁다.'

#사용 1 : 모든 형태소 보이기
text_analyzed = rhinoMorph.onlyMorph_list(rn, text)
print('\n1.형태소 분석 결과 : ',text_analyzed)

filepath:  /usr/local/lib/python3.7/dist-packages
classpath:  /usr/local/lib/python3.7/dist-packages/rhinoMorph/lib/rhino.jar
JVM is already started~
RHINO started!

1.형태소 분석 결과 :  ['한글', '로', '되', 'ㄴ', '한글', '텍스트', '를', '분석', '하', '는', '것', '은', '즐겁', '다', '.']


In [7]:
#사용 2 : 실질 형태소만, 동사의 어말어미는 제외
text_analyzed = rhinoMorph.onlyMorph_list(rn, text, pos=['NNG','NNP','NP','VV','VA','XR','IC','MM','MAG','MAJ'])
print('\n2.형태소 분석 결과 : ',text_analyzed)


2.형태소 분석 결과 :  ['한글', '되', '한글', '텍스트', '분석', '즐겁']


In [8]:
#사용 3 :실질 형태소만, 동사의 어말어미 포함
text_analyzed = rhinoMorph.onlyMorph_list(rn, text, pos=['NNG','NNP','NP','VV','VA','XR','IC','MM','MAG','MAJ'],eomi=True)
print('\n3.형태소 분석 결과 : ',text_analyzed)


3.형태소 분석 결과 :  ['한글', '되다', '한글', '텍스트', '분석', '즐겁다']


In [11]:
#사용 4 : 전체 형태소, 품사정보도 가져오기
morphs, poses = rhinoMorph.wholeResult_list(rn, text)
print('\n4. 형태소 분석 결과 : ')
print('morphs : ',morphs)
print('poses :', poses)


4. 형태소 분석 결과 : 
morphs :  ['한글', '로', '되', 'ㄴ', '한글', '텍스트', '를', '분석', '하', '는', '것', '은', '즐겁', '다', '.']
poses : ['NNG', 'JKB', 'VV', 'ETM', 'NNG', 'NNG', 'JKO', 'XR', 'XSV', 'ETM', 'NNB', 'JX', 'VA', 'EF', 'SF']


In [12]:
#사용 5 : 원문의 어절 정보를 같이 가져 오기
text_analyzed = rhinoMorph.wholeResult_list(rn, text)
print('\n5.형태소 분석 결과 : ',text_analyzed)


5.형태소 분석 결과 :  (['한글', '로', '되', 'ㄴ', '한글', '텍스트', '를', '분석', '하', '는', '것', '은', '즐겁', '다', '.'], ['NNG', 'JKB', 'VV', 'ETM', 'NNG', 'NNG', 'JKO', 'XR', 'XSV', 'ETM', 'NNB', 'JX', 'VA', 'EF', 'SF'])


### 감성분석

* 문서에서 긍정적 단어가 나타나면 +1, 부정적 단어가 나타나면 -1  
* 감성 점수 > 0 : 긍정적인 문서  
  감성 점수 < 0 : 부정적인 문서  
  감성 점수 = 0 : 중립적인 문서
*sigmoid 함수를 사용하여 값을 0~1 사이로 정규화하는 방법도 있음.

In [13]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [16]:
#파일이 있는 곳으로 경로를 변경한다.
%cd/content/gdrive/My\ Drive/pytest/

/content/gdrive/My Drive/pytest


In [29]:
#데이터 로딩
def read_data(filename, encoding='cp949'):  #읽기 함수 정의 , cp949 ??
  with open(filename, 'r', encoding=encoding) as f:
    data = [line.split('\t') for line in f.read().splitlines()]  
    #txt 파일의 헤더(id document label)는 제외하기
    #문장 분리-> 각 문장에서 탭 부분 분리 -> 리스트 생성 ->전체로는 중첩 리스트
    data=data[1:]
  return data

def write_data(data, filename, encoding='cp949'):  #쓰기 함수도 정의 
  with open(filename, 'w', encoding=encoding) as f:
    f.write(data)

data = read_data('ratings.txt', encoding='cp949')  #전체 파일은 ratings.txt

In [30]:
#전체 데이터 형태소 분석
import rhinoMorph
rn = rhinoMorph.startRhino()

morphed_data = ''

for data_each in data:
  morphed_data_each = rhinoMorph.onlyMorph_list(rn, data_each[1], pos=['NNG','NNP','VV','VA','XR','IC','MM','MAG','MAJ'], eomi=True)
  joined_data_each = ' '.join(morphed_data_each)
  if joined_data_each:
    morphed_data += data_each[0]+'\t'+joined_data_each+'\t'+data_each[2]+'\n'

#형태소 분석된 파일 저장
write_data(morphed_data, 'ratings_morphed.txt', encoding='cp949')

filepath:  /usr/local/lib/python3.7/dist-packages
classpath:  /usr/local/lib/python3.7/dist-packages/rhinoMorph/lib/rhino.jar
JVM is already started~
RHINO started!


In [32]:
#형태분석된 데이터 로딩
data = read_data('ratings_morphed.txt',encoding='cp949')
print(len(data))
print(len(data[0]))
print(data[0])

491
3
['4655635', '폴리스스토리 시리즈 뉴 없다 최고', '1']


In [33]:
#감성사전 읽기
data_id = [line[0] for line in data]  #데이터 id
data_text = [line[1] for line in data]  #데이터 본문
data_senti = [line[2] for line in data]  #데이터 긍부정 부분

positive = read_data('positive.txt')  #긍정 감성사전 읽기
negative = read_data('negative.txt')  #부정 감성사전 읽기

pos_fount=[]
neg_fount=[]

In [35]:
#감정 단어 파악
def cntWordInLine(data, senti):
  senti_fount=[]
  for onedata in data:
    oneline_word=onedata.split(' ') #한줄의 데이터를 공백 단위로 분리하여 리스트로 저장
    senti_temp=0 #그 줄에서 발견된 감정단어의 수를 담는 변수
    for sentiword in senti: #감정사전 어휘
      if sentiword[0] in oneline_word: #sentiword[0]하여 리스트 원소를 문자열로 추출
        senti_temp+=1  #현재의 감정단어와 일치하면 숫자를 하나 올려줌 (중복 방지)
    senti_fount.append(senti_temp)  #현재의 줄에서 찾은 감성단어의 숫자를 해당 위치에 저장
  return senti_fount

data_senti_poscnt = cntWordInLine(data_text, positive) #발견된 긍정 단어의 숫자 파악
data_senti_negcnt = cntWordInLine(data_text, negative) #발견된 부정 단어의 숫자 파악

print(data_senti_poscnt[:20])
print(data_senti_negcnt[:20])

[1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 1, 0, 1, 0, 1]
[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]


In [37]:
#감정 점수 계산
#pandas dataframe으로 저장
import pandas as pd
newdata = pd.DataFrame({'id':data_id,'text':data_text,'original':data_senti,'pos':data_senti_poscnt,'neg':data_senti_negcnt})
senti_score=newdata['pos']-newdata['neg']
newdata['senti_score']=senti_score

newdata.loc[newdata.senti_score>0,'new']=1  #새로운 긍부정 기호
newdata.loc[newdata.senti_score<=0,'new']=0 #새로운 긍부정 기호

#처음에 기록된 긍부정과 새로 계산된 긍부정이 같으니 여부를 matched칼럼에 저장
#original 컬럼은 문자로 되어 있으므로 숫자로 변환 뒤 비교
newdata.loc[pd.to_numeric(newdata.original)==newdata.new,'matched']='True'
newdata.loc[pd.to_numeric(newdata.original)!=newdata.new,'matched']='False'

In [38]:
#원점수와 비교 및 저장
score = newdata.matched.str.count('True').sum() / (newdata.matched.str.count('True').sum() + newdata.matched.str.count('False').sum()) * 100
print(score)

59.67413441955194


In [39]:
#저장
newdata.to_csv('newfile.csv', sep=',', encoding='cp949', index=False) #csv 저장
newdata.to_csv('newfile.txt',sep='\t', encoding='cp949', index=False) #txt저장

In [40]:
#시그모이드 점수 계산
import math
def sigmoid(x):
  return 1/(1+math.exp(-x))

newdata['sigmoid']=newdata.senti_score.apply(sigmoid)

In [41]:
#결과 확인
newdata.head()

,id,text,original,pos,neg,senti_score,new,matched,sigmoid
0,4655635,폴리스스토리 시리즈 뉴 없다 최고,1,1,0,1,1.0,True,0.731059
1,9251303,와 연기 진짜 쩔다 지루 생각 몰입 그래 이런 진짜 영화,1,0,1,-1,0.0,False,0.268941
2,10067386,안개 자욱 하다 밤하늘 뜨다 초승달 같다 영화,1,0,0,0,0.0,False,0.500000
3,2190435,사랑 해 사람 처음 끝 웃다 있다 영화,1,1,0,1,1.0,True,0.731059
4,9279041,완전 감동 다시다 감동,1,1,0,1,1.0,True,0.731059
